#Importante:
* Es ideal utilizar imagenes en alta definición
* Si quiere ejecutar el codigo es necesario que el entorno de ejecución sea la GPU

In [ ]:
#Instalamos mediapipe para realizar la segmentación de imagenes
!pip install mediapipe
#Clonamos el repositorio de "PIFUHD"
!git clone https://github.com/facebookresearch/pifuhd

     |████████████████████████████████| 32.7 MB 130 kB/s 
Cloning into 'pifuhd'...
remote: Enumerating objects: 213, done.
remote: Total 213 (delta 0), reused 0 (delta 0), pack-reused 213
Receiving objects: 100% (213/213), 402.72 KiB | 3.60 MiB/s, done.
Resolving deltas: 100% (104/104), done.


In [ ]:
%cd /content
#Clonamos el repositorio de "lightweight" que nos ayudará a obtener el esqueleto de nuestra imagen
!git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git
#Nos dirigimos dentro del directorio "lightweight"
%cd /content/lightweight-human-pose-estimation.pytorch/
#Descargamos el modelo pre-entrenado de "lightweight"
!wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth

#Ejecutamos el siguiente codigo que realiza un reshape de la resolución de la imagen que le pasaremos
#ya que alparecer 250 es la maxima resolución con la que se peude entrenar en Google Colab 
import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo

def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)
            
            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

#Nos dirigimos dentro del directorio "pifuhd"
%cd /content/pifuhd/
#Descargamos el modelo pre-entrenado de conversiones 2D a 3D
!sh ./scripts/download_trained_model.sh

/content
Cloning into 'lightweight-human-pose-estimation.pytorch'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 120 (delta 1), reused 0 (delta 0), pack-reused 115
Receiving objects: 100% (120/120), 227.79 KiB | 544.00 KiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/lightweight-human-pose-estimation.pytorch
--2022-03-31 02:03:17--  https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth
Resolving download.01.org (download.01.org)... 23.194.168.145, 2600:1407:3c00:1480::4b21, 2600:1407:3c00:1487::4b21
Connecting to download.01.org (download.01.org)|23.194.168.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87959810 (84M) [application/octet-stream]
Saving to: ‘checkpoint_iter_370000.pth’

checkpoint_iter_370 100%[===================>]  83.88M   158MB/s    in 0.5s    

2022-03-31 02:03:18 

#Pasamos al script de transformación 2D a 3D

In [ ]:
#Nos dirigimos al directorio "content"
%cd /content/

'''
¡¡ATENCION!! 
  *El programa presenta mejores resultados con imagenes en alta resolución
  *Para obtener siluetas correctas es recomendable recogerse el cabello largo antes de tomarse la foto y así dejar ver la medida de los hombros

1.Utilizar versiones anteriores a Python 3.10 y mayores iguales que Python 3.7
2.Instalar ndimage.whl segun las caracteristicas de su OS del siguiente enlace https://www.lfd.uci.edu/~gohlke/pythonlibs/#ndimage 
3.De la siguiente manera podrá instalar en el terminal el archivo .whl --- $ pip install "nombre del archivo".whl
4.Finalmente, instale las liberías correspondientes numpy,OpenCV,math,mediapipe y scipy
'''
#Importamos las librerías necesarias
import cv2
import matplotlib.pyplot as plt
import numpy as np
import mediapipe as mp
import os
from scipy.ndimage.morphology import binary_erosion

#Obtenemos la clase selfie_segmentation, la cual nos ayudará a realizar la segmentación
segmentation=mp.solutions.selfie_segmentation


#Incializamos un contexto "with" y utilizamos los valores configurables de SelfieSegmentation
#Tenemos los valores de 0 ó 1 como elección, en este caso escogeremos 1
#ya que nos permite de un modelo de rango completo mejor figuras dentro de los 5 metros
with segmentation.SelfieSegmentation(model_selection=1) as selfie_segmentation:
  #Inicializamos la imagen con OpenCV
  '''
  ACAAAAA PONEMOS LA RUTA DE LA IMAGEN QUE QUEREMOS SEGEMENTAR Y LUEGO CONVERTIRLO A 3D!!!!!!!!!!!!!!!!!!!!!!!!!!!
  '''
  im=cv2.imread("/content/sample_data/prueba_2.png")
  im_rgb = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
  
  #Convertimos nuestra imagen en formato BGR a RGB y procesamos con  MediaPipe Selfie Segmentation 
  im_result=selfie_segmentation.process(im_rgb)

    #A la imagen resultante obtendremos su UMBRAL, donde si los valores dentro de la imagen son mayores 
  #que 0.75 se visualizaran en blanco (255) y los menores en negro(0), para esto tambien le 
  #agregamos que queremos hacer una binarización binaria invertida, lo que resultará que tengamos
  #un fondo de color blanco
  _,tr_binary=cv2.threshold(im_result.segmentation_mask,0.75,255,cv2.THRESH_BINARY)
  
  #Una vez obtenida el umbral de la imagen, pasamos a realizarle una erosión donde rellenaremos los espacios faltantes en la imagen
  #El rellenador tendrá un tamaño de 1x1 y se realizaran 20 iteraciones donde buscará las partes por rellenar en la imagen
  eroded = binary_erosion(tr_binary, structure=np.ones((1, 1)), iterations=20)
  #Obtendremos una matriz con valores True y False
  #De la siguiente manera haremos el cambio de tipo de dato a float32
  image=np.uint8(eroded)
  #Despues de convertir la matriz de la imagen a valores tipo "uint8"
  #Cambiamos los valores diferentes a cero por 255 
  image[image!=0]=255
  #Realizamos un filtro medianBlur para suavizar la imagen
  tr_binary = cv2.medianBlur(image, 5)
 
  #Invertimos el threshold
  tr_binary_inv=cv2.bitwise_not(tr_binary)

  #Creamos una matriz con las mismaas dimensiones de nuestra imagen original
  bg_image = np.ones(im.shape,dtype=np.uint8)
  #Ah esa matriz le daremos un color de fondo
  bg_image[:]=(255,255,255)
  #Haremos un operacion AND (esta está por demás ya que lo hacemos con las dos imagenes)
  #Lo que nos ayuda de esta función es la posibilidad de poner una mascara
  #La forma de esta mascara será dada a la imagen que estamos colocando en la operación
  #Este serpa nuestro fondo
  bg = cv2.bitwise_and(bg_image,bg_image, mask=tr_binary_inv)

  #Haremos lo mismo aca pero con nuestra imagen de la persona y le pondremos la forma que obtuvimos
  #apartir de la segentación
  #Este será nuestra persona
  fg = cv2.bitwise_and(im, im, mask=tr_binary)

  #Sumamos las dos imagenes(fondo y la imagen segmentada)
  image_output=cv2.add(bg,fg)
  #Obtenemos la ruta donde se encuentra nuestra imagen 
  image_path="/content/pifuhd/sample_images/test.png"
  #Guardamos en la ruta de la carpeta pifuhd
  cv2.imwrite(image_path,image_output)

#Obtnemos la ruta del directorio de la imagen 
image_dir = os.path.dirname(image_path)
#Obtenemos el nombre del archivo imagen
file_name = os.path.splitext(os.path.basename(image_path))[0]
#Ponemos la ruta deseada de nuestro objeto
obj_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name

#Nos dirigimos al directorio "content/lightweight-human-pose-estimation.pytorch"
%cd /content/lightweight-human-pose-estimation.pytorch/

#Obtenemos el esqueleto de nuestra imagen
net = PoseEstimationWithMobileNet()
checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)

#Obtenemos el esqueleto de nuestra imagen
get_rect(net.cuda(), [image_path], 512)

#Nos dirigimos al directorio "/content/pifuhd/""
%cd /content/pifuhd/
#Ejecutamos el script de apps.simple_test con una resulución 256
#obteniendo datos de la ruta del directorio reservado en la variable image_dir
!python -m apps.simple_test -r 256 --use_rect -i $image_dir

/content
/content/lightweight-human-pose-estimation.pytorch


#Encuentre su resultado

In [ ]:
#La ruta del resultado es /content/pifuhd/results/pifuhd_final/recon/